In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import datetime
import os

import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras import layers, models
from tensorflow.keras.layers import LeakyReLU
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l1, l2, l1_l2
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [26]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

## Data Preprocessing

### User Segmented IMages

In [27]:
train_dataset = image_dataset_from_directory("C:\\Users\\adhim\\Desktop\\Compressed_Segmented1\\Train",
                                          image_size = (224, 224),
                                          batch_size = 32,
                                          shuffle = True
                                          )

Found 374 files belonging to 4 classes.


In [28]:
val_dataset = image_dataset_from_directory( "C:\\Users\\adhim\\Desktop\\Compressed_Segmented1\\Validation",
                                                image_size = (224, 224),
                                                batch_size = 32,
                                                shuffle = True)

Found 156 files belonging to 4 classes.


### Otsu Segmented

In [55]:
train_d = image_dataset_from_directory("C:\\Users\\adhim\\Desktop\\Compressed_Segmented3\\Train",
                                          image_size = (224, 224),
                                          batch_size = 32,
                                          shuffle = True
                                          )

Found 303 files belonging to 3 classes.


In [56]:
val_d = image_dataset_from_directory("C:\\Users\\adhim\\Desktop\\Compressed_Segmented3\\Validation",
                                          image_size = (224, 224),
                                          batch_size = 32,
                                          shuffle = True
                                          )

Found 131 files belonging to 3 classes.


## Configure the data

* **Use Buffered prefetching to load images from disk without having I/O become blocking.**

In [29]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size = AUTOTUNE)

val_dataset = val_dataset.prefetch(buffer_size = AUTOTUNE)

## Data Augmentation

In [30]:
data_augmentation = tf.keras.Sequential([
    
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

## Downloading Pretrained Model and Create a base model

In [31]:
preprocess_input = tf.keras.applications.resnet.preprocess_input

* **Create the base model from  the pre trained model ResNet50**

In [32]:
IMG_SHAPE = (224, 244) + (3,)

base_model = tf.keras.applications.ResNet50(input_shape = IMG_SHAPE,
                                            include_top = False,
                                            weights = 'imagenet')

* **Converting image pixel to model range pixel for compatibility.**

In [33]:
image_batch, label_batch = next(iter(train_dataset))

feature_batch = base_model(image_batch)

print(feature_batch.shape)

(32, 7, 7, 2048)


* **Frezzing th base model, prevents the weight in a given layer from being updated during training**

In [34]:
base_model.trainable = False

In [35]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 244, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 250, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 122, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 122, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

## Add a Classification Head

* **Averaging the 7X7 spatial locations, using tf.keras.layers.GlobalAveragPooling2D layer to convert the feature to a single single 2048-element vector per image.**

In [36]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

feature_batch_average = global_average_layer(feature_batch)

feature_batch_average.shape

TensorShape([32, 2048])

* **Apply a Dense layer to convert these feature into a single predictor per image.**

In [37]:
prediction_layer = layers.Dense(1)

prediction_batch = prediction_layer(feature_batch_average)

print(prediction_batch.shape)

(32, 1)


* **Create a pipeline**

In [44]:
inputs = tf.keras.Input( shape = (224, 224, 3) )

x = data_augmentation(inputs)

x = preprocess_input(x)

x = base_model(x, training = False)

x = global_average_layer(x)

outputs = prediction_layer(x)

model = tf.keras.Model(inputs, outputs)

## Compile the model

In [39]:
base_learning_rate = 0.0001

model.compile(optimizer = tf.keras.optimizers.Adam(lr = base_learning_rate),
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits =True),
              metrics = ['accuracy'])

In [40]:
model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 224, 224, 3)       0         
_________________________________________________________________
tf_op_layer_strided_slice_4  [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_BiasAdd_4 (Tenso [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)             

In [53]:
intial_epochs = 10

history = model.fit(train_dataset,
          epochs = intial_epochs,
          validation_data = val_dataset)

Epoch 1/10
12/12 [==============================] - 15s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 2/10
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 3/10
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 4/10
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 5/10
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 6/10
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 7/10
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss

#### Training Otsu segmented images

In [57]:
intial_epochs = 10

history = model.fit(train_d,
          epochs = intial_epochs,
          validation_data = val_d)

Epoch 1/10
10/10 [==============================] - 14s 1s/step - loss: 0.0000e+00 - accuracy: 0.3069 - val_loss: 0.0000e+00 - val_accuracy: 0.1832
Epoch 2/10
10/10 [==============================] - 10s 1s/step - loss: 0.0000e+00 - accuracy: 0.3069 - val_loss: 0.0000e+00 - val_accuracy: 0.1832
Epoch 3/10
10/10 [==============================] - 11s 1s/step - loss: 0.0000e+00 - accuracy: 0.3069 - val_loss: 0.0000e+00 - val_accuracy: 0.1832
Epoch 4/10
10/10 [==============================] - 10s 1s/step - loss: 0.0000e+00 - accuracy: 0.3069 - val_loss: 0.0000e+00 - val_accuracy: 0.1832
Epoch 5/10
10/10 [==============================] - 10s 1s/step - loss: 0.0000e+00 - accuracy: 0.3069 - val_loss: 0.0000e+00 - val_accuracy: 0.1832
Epoch 6/10
10/10 [==============================] - 11s 1s/step - loss: 0.0000e+00 - accuracy: 0.3069 - val_loss: 0.0000e+00 - val_accuracy: 0.1832
Epoch 7/10
10/10 [==============================] - 11s 1s/step - loss: 0.0000e+00 - accuracy: 0.3069 - val_loss

# Fine Tuning

In [45]:
base_model.trainable = True

In [47]:
# Number of layers in the base model

print("Number of layers in the base model ", len(base_model.layers))

Number of layers in the base model  175


In [48]:
# Fine tune from this layer

fine_tune_at = 100

In [49]:
# Freeze all layers before the 'fine_tune_at' layer

for layers in base_model.layers[:fine_tune_at]:
    
    layers.trainable = False

In [50]:
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop( lr = base_learning_rate/10 ),
              metrics = ['accuracy'])

In [51]:
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 224, 224, 3)       0         
_________________________________________________________________
tf_op_layer_strided_slice_7  [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_BiasAdd_7 (Tenso [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 8, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

In [54]:
fine_tune_epochs =10

total_epochs = intial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=  total_epochs,
                         initial_epoch = history.epoch[-1],
                         validation_data = val_dataset)

Epoch 10/20
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 11/20
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 12/20
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 13/20
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 14/20
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 15/20
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - val_loss: 0.0000e+00 - val_accuracy: 0.1218
Epoch 16/20
12/12 [==============================] - 13s 1s/step - loss: 0.0000e+00 - accuracy: 0.0882 - v